In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#### 读入数据

In [4]:
model = pd.read_csv("C:\\Users\\chenxiaoyan\\Desktop\\model.csv"

#### 所有小于0的数处理为np.nan

In [5]:
def rep(x):
    """ 查找数值型的小于0的值并置为空 """
    if type(x) !=str and x < 0:
        return np.nan
    return x

In [6]:
model_df = model.applymap(rep)
model_df.shape

(13890, 1269)

#### 计算变量缺失率并且将缺失率大于0.75的变量删除

In [7]:
def Missing_rate(x):
    """计算每一个变量的缺失率"""
    Missing_rate = 1-x.count()/len(x)
    return Missing_rate

In [8]:
model_df.apply(Missing_rate,axis = 0)#drop掉缺失率大于75%的变量
model_df  = model_df.drop(list(model_df.apply(Missing_rate,axis = 0)[model_df.apply(Missing_rate,axis = 0)>=0.8].index),axis = 1)
model_df.shape

(13890, 845)

In [9]:
a = model_df.apply(lambda x : 1-x.count()/len(x),axis = 0) #计算缺失率的lambda表达式

#### 单一度-变量取值最大占比

In [10]:
def max_percent(x):
    max_percent = x.value_counts(normalize=True,dropna=False).max()
    return max_percent

In [11]:
model_df=model_df.drop(list(model_df.apply(max_percent,axis = 0)[model_df.apply(max_percent,axis = 0)>=0.9].index),axis = 1)
model_df.shape

(13890, 756)

#### 划分训练集和测试集

In [12]:
train_ ,test=train_test_split(model_df,test_size=0.2,stratify =np.array(model_df['openmonth'],model_df['default']),random_state = 1)

In [13]:
print(train_.shape,test.shape)

(11112, 756) (2778, 756)


In [14]:
train = train_[~train_['default'].isin([0.5,2])]#删除掉default=0.5和2的行
test['default']  = test['default'].replace({0.5:0,2:1})#将test数据集中的0.5换成0,2换成1

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
print(train['default'].value_counts(normalize=True,dropna=False))
print(test['default'].value_counts(normalize=True,dropna=False))

0.0    0.888426
1.0    0.111574
Name: default, dtype: float64
0.0    0.861411
1.0    0.138589
Name: default, dtype: float64


In [16]:
print(train['default'].value_counts(dropna=False))

0.0    8743
1.0    1098
Name: default, dtype: int64


#### 字符型分箱

In [17]:
def char_bin(x):
    """字符型变量分箱"""
    total_char = pd.DataFrame()
    for char in x:
        g = train.groupby(train[char])
        n = len(train)
        good_n = len(train[train['default'].isin([0.0])])
        bad_n = len(train[train['default'].isin([1.0])])
        value = list(g.groups.keys())

        var_name = []
        bin_n = []
        bin_good_n = []
        bin_bad_n = []
        bin_rate = []
        bad_rate = []
        good_rate = []
        woe = []
        pre_iv = []
        iv = []
        for i in range(0,len(g.groups)):
            
            var_name1 =char
            bin_n1 = len(g.get_group(value[i]))
            bin_good_n1 = len(g.get_group(value[i])[g.get_group(value[i])['default'].isin([0.0])])
            bin_bad_n1 = len(g.get_group(value[i])[g.get_group(value[i])['default'].isin([1.0])])
            bin_rate1 = '{:.2%}'.format(bin_bad_n1/bad_n)
            bad_rate1 = '{:.2%}'.format(bin_bad_n1/bin_n1)
            good_rate1 = '{:.2%}'.format(bin_good_n1/bin_n1)
            
            woe1 = np.log((bin_bad_n1/bad_n)/(bin_good_n1/good_n))
       
            pre_iv1 = woe1 * (bin_bad_n1/bad_n)/(bin_good_n1/good_n)
            
    
            bin_n.append(bin_n1)
            bin_good_n.append(bin_good_n1)
            bin_bad_n.append(bin_bad_n1)
            var_name.append(var_name1)
            bin_rate.append(bin_rate1)
            bad_rate.append(bad_rate1)
            good_rate.append(good_rate1)
            woe.append(woe1)
            pre_iv.append(pre_iv1)   
            iv1 = sum(pre_iv)
            iv.append(iv1)
            
            
                
        cols = ['var_name','value','bin_n','bin_good_n','bin_bad_n','bin_rate','bad_rate','woe','pre_iv','iv']
        char1 = pd.DataFrame(np.array([var_name,value,bin_n,bin_good_n,bin_bad_n,bin_rate,bad_rate,woe,pre_iv,iv]).T,columns =cols)
        total_char= total_char.append(char1,ignore_index=False)
    return total_char

In [18]:
char_bin(['education','marriage','show_first_name','marriage_gender_status','age_gender_status','house_product_type','show_first_name','gender','marriage_age_gender_status','marriage_age_gender_status'])
#char_bin(['marriage_age_gender_status'])
#解决除数为0的方法 ；log(0)  的方法  - 用np.log() 
#def func(x):
 #   try:
  #      return 1 / x
   # except ZeroDivisionError:
    #    return np.nan

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


,var_name,value,bin_n,bin_good_n,bin_bad_n,bin_rate,bad_rate,woe,pre_iv,iv
0,education,1,61,57,4,0.36%,6.56%,-0.5819938696032445,-0.32520834703870943,-0.32520834703870943
1,education,2,1698,1552,146,13.30%,8.60%,-0.2889300419234126,-0.21642726821394678,-0.5416356152526562
2,education,3,5902,5250,652,59.38%,11.05%,-0.011175756547601612,-0.011051554332758076,-0.5526871695854143
3,education,4,2014,1745,269,24.50%,13.36%,0.20496458207672677,0.2515902509668766,-0.3010969186185377
4,education,5,166,139,27,2.46%,16.27%,0.4361259699850522,0.6745576168615364,0.37346069824299866
0,marriage,1,2352,2099,253,23.04%,10.76%,-0.041063793981697815,-0.03941171133961872,-0.03941171133961872
1,marriage,2,6414,5753,661,60.20%,10.31%,-0.08895985990019206,-0.08138780090437929,-0.12079951224399801
2,marriage,3,1016,841,175,15.94%,17.22%,0.504957351061981,0.8366713352551147,0.7158718230111167
3,marriage,4,30,24,6,0.55%,20.00%,0.6884686759915244,1.3705103902991569,2.086382213310274
4,marriage,5,29,26,3,0.27%,10.34%,-0.08472121224195737,-0.07783917177715777,2.008543041533116


#### 离散型变量分箱

##### 等频分箱

In [19]:
train.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_petLnQry_2y,td_petLnQry_3m,td_petLnQry_6m,td_petLnQry_7d,td_propInsQry_18m,td_propInsQry_1y,td_propInsQry_2y,td_propInsQry_3m,td_propInsQry_6m,openmonth
13331,0.0,GD1811010810015950,2.0,NaN,NaN,NaN,NaN,1,1,0,...,4.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,43252
12352,0.0,GD1837020201095650,2.0,NaN,NaN,NaN,NaN,2,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
12148,0.0,GD1850010702077450,2.0,NaN,NaN,NaN,NaN,0,0,0,...,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
2645,0.0,GD1833048102034870,NaN,1.0,1.0,1.0,1.0,1,1,1,...,7.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,43221
11299,1.0,GD1833010401070330,NaN,2.0,7.0,2.0,7.0,0,0,0,...,5.0,5.0,5.0,4.0,1.0,1.0,1.0,0.0,1.0,43252


In [20]:
train_bin = train.copy() #train_bin初始分箱的数据集
train_bin.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_petLnQry_2y,td_petLnQry_3m,td_petLnQry_6m,td_petLnQry_7d,td_propInsQry_18m,td_propInsQry_1y,td_propInsQry_2y,td_propInsQry_3m,td_propInsQry_6m,openmonth
13331,0.0,GD1811010810015950,2.0,NaN,NaN,NaN,NaN,1,1,0,...,4.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,43252
12352,0.0,GD1837020201095650,2.0,NaN,NaN,NaN,NaN,2,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
12148,0.0,GD1850010702077450,2.0,NaN,NaN,NaN,NaN,0,0,0,...,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
2645,0.0,GD1833048102034870,NaN,1.0,1.0,1.0,1.0,1,1,1,...,7.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,43221
11299,1.0,GD1833010401070330,NaN,2.0,7.0,2.0,7.0,0,0,0,...,5.0,5.0,5.0,4.0,1.0,1.0,1.0,0.0,1.0,43252


In [21]:
def pct_rank_qcut(series,n):
   
    """离散型变量分箱"""
    edages = pd.Series([i/10 for i in range(n)]) #生成分位数
    
    func = lambda x:(edages >= x).idxmax() #argmax()返回最大值对应的索引,索引从0开始计算； func函数：输入x输出，输出百分位数对应的索引值
    train_bin[series + '_bin' ] = train_bin[series].rank(na_option = 'top',pct=True).astype(float).apply(func)
    return train_bin
    
    #rank()排序函数，每一个数在向量中的排序，相同的数，取平均；na_option：缺失值的排名处理；pct:布尔型，默认以1开始的数输出排名，pct = True 以百分比的排名输出                                               

In [22]:
for i in (train_bin.columns.drop(['default','id'])):
    pct_rank_qcut(i,11)

In [23]:
print (train_bin.shape)
print (train.shape)
train_bin.head()
train_bin.to_csv("C:\\Users\\chenxiaoyan\\Desktop\\a.csv")

(9841, 1510)
(9841, 756)


In [24]:
train_bin.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_petLnQry_2y_bin,td_petLnQry_3m_bin,td_petLnQry_6m_bin,td_petLnQry_7d_bin,td_propInsQry_18m_bin,td_propInsQry_1y_bin,td_propInsQry_2y_bin,td_propInsQry_3m_bin,td_propInsQry_6m_bin,openmonth_bin
13331,0.0,GD1811010810015950,2.0,NaN,NaN,NaN,NaN,1,1,0,...,8,6,9,5,9,9,8,5,9,8
12352,0.0,GD1837020201095650,2.0,NaN,NaN,NaN,NaN,2,0,0,...,2,3,2,2,4,4,4,5,4,8
12148,0.0,GD1850010702077450,2.0,NaN,NaN,NaN,NaN,0,0,0,...,5,6,5,2,4,4,4,5,4,8
2645,0.0,GD1833048102034870,NaN,1.0,1.0,1.0,1.0,1,1,1,...,10,10,10,10,4,4,4,5,4,3
11299,1.0,GD1833010401070330,NaN,2.0,7.0,2.0,7.0,0,0,0,...,9,10,10,10,9,9,8,5,9,8


In [25]:
def num_bin(indata,var_old,var_new):
    """连续型变量分箱"""
    total_num = pd.DataFrame()
    for num,num1 in zip(var_new,var_old):
        g = indata[[num1,num,'default']].groupby(indata[num])
        n = len(indata)
        good_n = len(indata[train_bin['default'].isin([0.0])])
        bad_n = len(indata[indata['default'].isin([1.0])])
        value_ = list(g.max()[num1])
        value = list(g.groups.keys())
        
        var_name = []
        bin_n = []
        bin_good_n = []
        bin_bad_n = []
        bin_rate = []
        bad_rate = []
        good_rate = []
        woe = []
        pre_iv = []
        iv = []
        for i in range(0,len(g.groups)):
            
            var_name1 =num
            bin_n1 = len(g.get_group(value[i]))
            bin_good_n1 = len(g.get_group(value[i])[g.get_group(value[i])['default'].isin([0.0])])
            bin_bad_n1 = len(g.get_group(value[i])[g.get_group(value[i])['default'].isin([1.0])])
            bin_rate1 = '{:.2%}'.format(bin_bad_n1/bad_n)
            bad_rate1 = '{:.2%}'.format(bin_bad_n1/bin_n1)
            good_rate1 = '{:.2%}'.format(bin_good_n1/bin_n1)
            
            woe1 = np.log((bin_bad_n1/bad_n)/(bin_good_n1/good_n))
       
            pre_iv1 = woe1 * (bin_bad_n1/bad_n)/(bin_good_n1/good_n)
            
    
            bin_n.append(bin_n1)
            bin_good_n.append(bin_good_n1)
            bin_bad_n.append(bin_bad_n1)
            var_name.append(var_name1)
            bin_rate.append(bin_rate1)
            bad_rate.append(bad_rate1)
            good_rate.append(good_rate1)
            woe.append(woe1)
            pre_iv.append(pre_iv1)   
            iv1 = sum(pre_iv)
            iv.append(iv1)
            
            
                
        cols = ['var_name','value','value_','bin_n','bin_good_n','bin_bad_n','bin_rate','bad_rate','woe','pre_iv','iv']
        num1 = pd.DataFrame(np.array([var_name,value_,value,bin_n,bin_good_n,bin_bad_n,bin_rate,bad_rate,woe,pre_iv,iv]).T,columns =cols)
        total_num= total_num.append(num1,ignore_index=False)
    return total_num

In [26]:
train_bin.head()
#y = train_bin.iloc[:,2:756].columns
#x = train_bin.iloc[:,756:].columns
result  = num_bin(train_bin, train_bin.iloc[:,2:756].columns,train_bin.iloc[:,756:].columns)

In [27]:
result.to_csv("C:\\Users\\chenxiaoyan\\Desktop\\woe.csv")

#### 将手工分箱变量导入,映射新的分组

In [28]:
train1 = train.copy()#手工分箱后的数据集1
train1.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_petLnQry_2y,td_petLnQry_3m,td_petLnQry_6m,td_petLnQry_7d,td_propInsQry_18m,td_propInsQry_1y,td_propInsQry_2y,td_propInsQry_3m,td_propInsQry_6m,openmonth
13331,0.0,GD1811010810015950,2.0,NaN,NaN,NaN,NaN,1,1,0,...,4.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,43252
12352,0.0,GD1837020201095650,2.0,NaN,NaN,NaN,NaN,2,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
12148,0.0,GD1850010702077450,2.0,NaN,NaN,NaN,NaN,0,0,0,...,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
2645,0.0,GD1833048102034870,NaN,1.0,1.0,1.0,1.0,1,1,1,...,7.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,43221
11299,1.0,GD1833010401070330,NaN,2.0,7.0,2.0,7.0,0,0,0,...,5.0,5.0,5.0,4.0,1.0,1.0,1.0,0.0,1.0,43252


In [29]:
train1 = train1.fillna(-99)

In [30]:
train1.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_petLnQry_2y,td_petLnQry_3m,td_petLnQry_6m,td_petLnQry_7d,td_propInsQry_18m,td_propInsQry_1y,td_propInsQry_2y,td_propInsQry_3m,td_propInsQry_6m,openmonth
13331,0.0,GD1811010810015950,2.0,-99.0,-99.0,-99.0,-99.0,1,1,0,...,4.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,43252
12352,0.0,GD1837020201095650,2.0,-99.0,-99.0,-99.0,-99.0,2,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
12148,0.0,GD1850010702077450,2.0,-99.0,-99.0,-99.0,-99.0,0,0,0,...,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,43252
2645,0.0,GD1833048102034870,-99.0,1.0,1.0,1.0,1.0,1,1,1,...,7.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,43221
11299,1.0,GD1833010401070330,-99.0,2.0,7.0,2.0,7.0,0,0,0,...,5.0,5.0,5.0,4.0,1.0,1.0,1.0,0.0,1.0,43252


In [31]:
config = pd.read_excel("C:\\Users\\chenxiaoyan\\Desktop\\config.xlsx")
config

,varname,min,max,bin
0,in_month,-100,-1,0
1,in_month,0,20,1
2,in_month,21,119,2
3,in_month,120,99999,3
4,td_allQry_18m,-100,-1,0
5,td_allQry_18m,0,5,1
6,td_allQry_18m,6,13,2
7,td_allQry_18m,14,99999,3
8,td_allQry_1m,-100,-1,0
9,td_allQry_1m,0,3,1


In [32]:
for i in  list(config.varname.unique()):
    def func(a):
        min_ = config[config['varname'] == i]['min'].tolist()
        max_ = config[config['varname'] == i]['max'].tolist()
        bin_ = config[config['varname'] == i]['bin'].tolist()
        for min_,max_,bin_ in zip(min_,max_,bin_):
            if a>= min_ and a<= max_:
                return bin_
    train1[i+'_bin'] = train1[i].apply(lambda x : func(x))

In [33]:
train1.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,td_propInsQry_1y,td_propInsQry_2y,td_propInsQry_3m,td_propInsQry_6m,openmonth,in_month_bin,td_allQry_18m_bin,td_allQry_1m_bin,stab_mail_num_bin,m12_cell_cooff_allnum_bin
13331,0.0,GD1811010810015950,2.0,-99.0,-99.0,-99.0,-99.0,1,1,0,...,1.0,1.0,0.0,1.0,43252,2,3,2,0,1
12352,0.0,GD1837020201095650,2.0,-99.0,-99.0,-99.0,-99.0,2,0,0,...,0.0,0.0,0.0,0.0,43252,2,2,1,0,3
12148,0.0,GD1850010702077450,2.0,-99.0,-99.0,-99.0,-99.0,0,0,0,...,0.0,0.0,0.0,0.0,43252,3,2,2,1,0
2645,0.0,GD1833048102034870,-99.0,1.0,1.0,1.0,1.0,1,1,1,...,0.0,0.0,0.0,0.0,43221,2,3,3,1,1
11299,1.0,GD1833010401070330,-99.0,2.0,7.0,2.0,7.0,0,0,0,...,1.0,1.0,0.0,1.0,43252,2,3,3,0,1


#### 计算手工分箱之后的IV和woe

In [34]:
#此处可以修改
result = num_bin(train1,['in_month','td_allQry_18m','td_allQry_1m','stab_mail_num','m12_cell_cooff_allnum'] ,
                  ['in_month_bin','td_allQry_18m_bin','td_allQry_1m_bin','stab_mail_num_bin','m12_cell_cooff_allnum_bin'])

In [35]:
result

,var_name,value,value_,bin_n,bin_good_n,bin_bad_n,bin_rate,bad_rate,woe,pre_iv,iv
0,in_month_bin,-99.0,0,2358,2115,243,22.13%,10.31%,-0.0889856110284149,-0.08140926374831861,-0.08140926374831861
1,in_month_bin,20.0,1,594,495,99,9.02%,16.67%,0.4653251246773148,0.7410450938167147,0.6596358300683961
2,in_month_bin,119.0,2,4912,4320,592,53.92%,12.05%,0.08725899075726472,0.09521519995634595,0.7548510300247421
3,in_month_bin,311.0,3,1977,1813,164,14.94%,8.30%,-0.32810874581923755,-0.23633170544770538,0.5185193245770368
0,td_allQry_18m_bin,-99.0,0,18,16,2,0.18%,11.11%,-0.0046785045684208425,-0.004656667286168423,-0.004656667286168423
1,td_allQry_18m_bin,5.0,1,2015,1861,154,14.03%,7.64%,-0.41715361711720655,-0.2748707324176846,-0.27952739970385304
2,td_allQry_18m_bin,13.0,2,2972,2672,300,27.32%,10.09%,-0.11203702288892024,-0.10016234857312319,-0.3796897482769762
3,td_allQry_18m_bin,155.0,3,4836,4194,642,58.47%,13.28%,0.19794112933951094,0.2412685887961749,-0.1384211594808013
0,td_allQry_1m_bin,-99.0,0,18,16,2,0.18%,11.11%,-0.0046785045684208425,-0.004656667286168423,-0.004656667286168423
1,td_allQry_1m_bin,3.0,1,3871,3518,353,32.15%,9.12%,-0.22441683125279172,-0.17930501411477243,-0.18396168140094085


#### 将woe映射到训练数据集上

In [36]:
result['value_']=result.value_.astype('int64')

In [52]:
for i in  list(result.var_name.unique()):
    def func(a):
        bin_ = result[result['var_name'] == i]['value_'].tolist()
        woe_ = result[result['var_name'] == i]['woe'].tolist()
        for bin_,woe_ in zip(bin_,woe_):
            if a == bin_:
                return woe_
    
    train1[i+'_w'] = train1[i].apply(lambda x : func(x)).astype('float64')

In [53]:
train1.head()

,default,id,wdxh_flag_code,currentLoanOrgCount,currentLoanCount,m12LoanOrgCount,m12LoanCount,j1_repay_uno_num,j1_6m_judai_num,j1_3m_judai_num,...,in_month_bin,td_allQry_18m_bin,td_allQry_1m_bin,stab_mail_num_bin,m12_cell_cooff_allnum_bin,in_month_bin_w,td_allQry_18m_bin_w,td_allQry_1m_bin_w,stab_mail_num_bin_w,m12_cell_cooff_allnum_bin_w
13331,0.0,GD1811010810015950,2.0,-99.0,-99.0,-99.0,-99.0,1,1,0,...,2,3,2,0,1,0.087259,0.197941,0.092365,0.143742,0.111476
12352,0.0,GD1837020201095650,2.0,-99.0,-99.0,-99.0,-99.0,2,0,0,...,2,2,1,0,3,0.087259,-0.112037,-0.224417,0.143742,0.378149
12148,0.0,GD1850010702077450,2.0,-99.0,-99.0,-99.0,-99.0,0,0,0,...,3,2,2,1,0,-0.328109,-0.112037,0.092365,-0.026103,-0.178648
2645,0.0,GD1833048102034870,-99.0,1.0,1.0,1.0,1.0,1,1,1,...,2,3,3,1,1,0.087259,0.197941,0.277500,-0.026103,0.111476
11299,1.0,GD1833010401070330,-99.0,2.0,7.0,2.0,7.0,0,0,0,...,2,3,3,0,1,0.087259,0.197941,0.277500,0.143742,0.111476


In [54]:
train1.dtypes

default                        float64
id                              object
wdxh_flag_code                 float64
currentLoanOrgCount            float64
currentLoanCount               float64
m12LoanOrgCount                float64
m12LoanCount                   float64
j1_repay_uno_num                 int64
j1_6m_judai_num                  int64
j1_3m_judai_num                  int64
j1_6m_ok_num                     int64
cell                            object
user_date                       object
sl_user_date                    object
cus_num                          int64
flag_location                    int64
flag_netshopping                 int64
flag_stability_c                 int64
location_cell_attr             float64
location_cell_stab             float64
location_cell_use_days         float64
location_id_attr               float64
location_live_birth_mat        float64
location_loan_addr_num         float64
location_loan_baddr_num        float64
location_loan_haddr_num  

#### 逐步回归法选择变量

In [58]:
import statsmodels.formula.api as smf #引入类似R语言表达式模块接口
import statsmodels.api as sm

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import logging
import os
from scipy.stats import chi2
  def stepwise(self, X, y, logging_file, start_from=[], direction='FORWARD',
                    lrt=True, lrt_threshold=0.05):
        """
        Use results of logistic regression to select features. It will go through
        stepwise based on the direction specified first and then check the
        significance of the variables using pvalue. Then check if there are variables
        with negative coefficients and remove it.

        Args:
        X (pd.DataFrame()): X data frame that contains the variables.
            variable names should not contain . or -, replace with _ and _
        y (pd.Series()): y data, labeling the performance
        logging_file (str): logging file object
        start_from (list): list of variable names that the model starts from,
            if not the null model
        direction (str): default='FOWARD/BACKWARD', forward first and then backward.
            ['FORWARD/BACKWARD', 'BACKWARD/FORWARD']
        lrt (bool): default = True. If set true, the model will consider likelihood
            ratio test result
        lrt_threshold (float): default = 0.05. The alpha value for likelihood ratio
            test.

        Returns:
        imp_df (pd.DataFrame()): contains the variable names and FORWARD AIC
            and ranking0
        """
        def _likelihood_ratio_test(ll_r, ll_f, lrt_threshold):
            # H0: reduced model is true. If rejected, then it means that the
            # full model is good. Need to add the candidate
            test_statistics = (-2 * ll_r) - (-2 * ll_f)
            p_value = 1 - chi2(df=1).cdf(test_statistics)
            return p_value <= lrt_threshold




        def _forward(dataset, logging_file, current_score, best_new_score,
                    remaining, selected, result_dict, reduced_loglikelihood):
            # TODO: 加上 f test like in SAS
            logging_file.info("While loop beginning current_score: %s" % current_score)
            logging_file.info("While loop beginning best_new_score: %s" % best_new_score)
            current_score = best_new_score
            aics_with_candidates = {}
            p_values_ok_to_add = []
            # 选择最好的变量to add
            for candidate in remaining:
                formula = "{} ~ {}".format('y', ' + '.join(selected + [candidate]))
                mod1 = smf.glm(formula=formula, data=dataset, family=sm.families.Binomial()).fit()
                # 只有新加指标的coefficient>0或者加上之后其他指标coefficient也>0
                if sum(mod1.params.loc[mod1.params.index != 'Intercept'] < 0) == 0 :
                    aics_with_candidates[candidate] = mod1.aic
                    full_loglikelihood = mod1.llf
                    if lrt:
                        p_values_ok = _likelihood_ratio_test(reduced_loglikelihood, full_loglikelihood, lrt_threshold)
                        if p_values_ok:
                            p_values_ok_to_add.append(candidate)

            # 只有通过likelihood ratio test的变量才会进行AIC比较进行选择
            candidate_scores = pd.Series(aics_with_candidates)
            if lrt:
                candidate_scores = candidate_scores.loc[p_values_ok_to_add]

            # 有变量pvalues显著 reject the reduced model and need to add the variable
            if not candidate_scores.empty:
                best = candidate_scores[candidate_scores == candidate_scores.min()]
                # best_new_score 被替换成新的加上这个变量的模型的AIC
                best_new_score = best.iloc[0]
                best_candidate = best.index.values[0]
            else:
                return None

            # 当加上变量的模型的AIC比当前模型的小时，选择加上变量的模型
            if current_score > best_new_score:
                logging_file.info('Best Variable to Add: %s' % best_candidate)
                remaining.remove(best_candidate)
                selected.append(best_candidate)
                improvement_gained = current_score - best_new_score
                result_dict[best_candidate] = {'AIC_delta': improvement_gained, 'step': 'FORWARD'}
                formula = "{} ~ {}".format('y', ' + '.join(selected))
                mod2 = smf.glm(formula=formula, data=dataset, family=sm.families.Binomial()).fit()
                # loglikelihood of the reduced model
                reduced_loglikelihood = mod2.llf
                logging_file.info('FOWARD Step: AIC=%s' % mod2.aic)
                logging_file.info(mod2.summary())
                return current_score, best_new_score, result_dict, selected, remaining, reduced_loglikelihood
            else:
                return None



        def _backward(dataset, logging_file, current_score, best_new_score,
                    remaining, selected, result_dict, reduced_loglikelihood):
            # TODO: 加上 f test like in SAS
            logging_file.info("While loop beginning current_score: %s" % current_score)
            logging_file.info("While loop beginning best_new_score: %s" % best_new_score)
            current_score = best_new_score
            aics_with_candidates = {}
            p_values_ok_to_delete = []
            # 选择最差的to delete
            for candidate in selected:
                put_in_model = [i for i in selected if i != candidate]
                formula = "{} ~ {}".format('y', ' + '.join(put_in_model))
                mod1 = smf.glm(formula=formula, data=dataset, family=sm.families.Binomial()).fit()
                # 只有减去指标后留下的变量中没有变量coefficient是负数
                if sum(mod1.params.loc[mod1.params.index != 'Intercept'] < 0) == 0 :
                    aics_with_candidates[candidate] = mod1.aic
                    reduced_reduced_loglikelihood = mod1.llf
                    if lrt:
                        p_values_rejected = _likelihood_ratio_test(reduced_reduced_loglikelihood, reduced_loglikelihood, lrt_threshold)
                        # if not rejected, H0: reduced model is true, need to remove the variable
                        if not p_values_rejected:
                            p_values_ok_to_delete.append(candidate)

            # 只有通过likelihood ratio test的变量才会进行AIC比较进行选择
            candidate_scores = pd.Series(aics_with_candidates)
            if lrt:
                candidate_scores = candidate_scores.loc[p_values_ok_to_delete]

            # 有变量pvalues不显著 没有reject the reduced model, then need to delete the variable
            if not candidate_scores.empty:
                best = candidate_scores[candidate_scores == candidate_scores.min()]
                # best_new_score 被替换成新的减去这个变量的模型的AIC
                best_new_score = best.iloc[0]
                best_candidate = best.index.values[0]
            else:
                return None


            # 当减去变量的模型的AIC比当前模型的小时，选择减去变量的模型
            if current_score > best_new_score:
                logging_file.info('Best Variable to Delete: %s' % best_candidate)
                remaining.append(best_candidate)
                selected.remove(best_candidate)
                improvement_gained = current_score - best_new_score
                result_dict[best_candidate] = {'AIC_delta': improvement_gained, 'step': 'BACKWARD'}
                formula = "{} ~ {}".format('y', ' + '.join(selected))
                mod2 = smf.glm(formula=formula, data=dataset, family=sm.families.Binomial()).fit()
                # loglikelihood of the reduced model
                reduced_loglikelihood = mod2.llf
                logging_file.info('BACKWARD Step: AIC=%s' % mod2.aic)
                logging_file.info(mod2.summary())
                return current_score, best_new_score, result_dict, selected, remaining, reduced_loglikelihood
            else:
                return None

#### 逻辑回归拟合模型

In [56]:
lg = smf.glm('default ~ in_month_bin_w + td_allQry_18m_bin_w + td_allQry_1m_bin_w + stab_mail_num_bin_w + m12_cell_cooff_allnum_bin_w +in_month', 
             data=train1, family=sm.families.Binomial(sm.families.links.logit)).fit()
lg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                 9841
Model:                            GLM   Df Residuals:                     9834
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -3371.4
Date:                Thu, 26 Jul 2018   Deviance:                       6742.7
Time:                        15:09:41   Pearson chi2:                 9.83e+03
No. Iterations:                     5                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -2.0681      0.036    -57.440      0.000      -2.139      -1.998
in_month_bin_w                  0.8676      0.165      5.256      0.000       0.544       1.191
td_allQry_18m_bin_w             0.8353      0.179      4.657      0.000       0.484       1.187
td_allQry_1m_bin_w              0.3252      0.225      1.447      0.148      -0.115       0.766
stab_mail_num_bin_w             1.1399      0.214      5.334      0.000       0.721       1.559
m12_cell_cooff_allnum_bin_w     0.6573      0.166      3.964      0.000       0.332       0.982
in_month                       -0.0002      0.000     -0.431      0.666      -0.001       0.001
===============================================================================================
"""

In [57]:
#sm.Logit()